# BAX pore assembly

In [ ]:
from scripts.reference_solutions.ssa_helper import SSASol
from output_helper import *
%matplotlib inline

In [ ]:
with xr.open_dataset("output/bax_general_230730_r5_so_tau1e0_ss100/output_t100.nc") as ds:
    grid = GridInfo(ds)
    lr_sol = LRSol(ds, grid)
    slice_vec = np.array([17, 9, 2, 1, 0, 0, 0, 0, 0, 50, 0])
    idx_2D = np.array([0, 1])
    P_marginal = lr_sol.marginalDistributions()
    P_marginal2D = lr_sol.marginalDistribution2D(idx_2D)
    P_sliced = lr_sol.slicedDistributions(slice_vec)
    P_sliced2D = lr_sol.slicedDistribution2D(slice_vec, idx_2D)

## Figure 1
Sliced distribution $P_\mathrm{S}(x_1) = P(x_1, 9, 2, 1, 0, 0, 0, 0, 0, 50, 0)$ for $t = 145$

In [ ]:
result = np.load("scripts/reference_solutions/bax_ssa_general_ref_t145_1e4.npy")
ssa_sol_1e4 = SSASol(result)
P_marginal_ssa_1e4, _, P_sliced_ssa_1e4, _ = ssa_sol_1e4.calculateObservables(slice_vec, idx_2D)

result = np.load("scripts/reference_solutions/bax_ssa_general_ref_t145_1e5.npy")
ssa_sol_1e5 = SSASol(result)
P_marginal_ssa_1e5, _, P_sliced_ssa_1e5, _ = ssa_sol_1e5.calculateObservables(slice_vec, idx_2D)

result = np.load("scripts/reference_solutions/bax_ssa_general_ref_t145_1e6.npy")
ssa_sol_1e6 = SSASol(result)
P_marginal_ssa_1e6, _, P_sliced_ssa_1e6, _ = ssa_sol_1e6.calculateObservables(slice_vec, idx_2D)

result = np.load("scripts/reference_solutions/bax_ssa_general_ref_t145_1e7.npy")
ssa_sol_1e7 = SSASol(result)
P_marginal_ssa_1e7, _, P_sliced_ssa_1e7, _ = ssa_sol_1e7.calculateObservables(slice_vec, idx_2D)

In [ ]:
mesh_dlr = grid.bin[0] * np.arange(grid.n[0]) + grid.liml[0]
mesh_ssa_1e4 = np.arange(ssa_sol_1e4.n[0]) + ssa_sol_1e4.n_min[0]
mesh_ssa_1e5 = np.arange(ssa_sol_1e5.n[0]) + ssa_sol_1e5.n_min[0]
mesh_ssa_1e6 = np.arange(ssa_sol_1e6.n[0]) + ssa_sol_1e6.n_min[0]
mesh_ssa_1e7 = np.arange(ssa_sol_1e7.n[0]) + ssa_sol_1e7.n_min[0]

P_sliced_ssa_1e4_red = np.interp(mesh_dlr, mesh_ssa_1e4, P_sliced_ssa_1e4[-1][0])
P_sliced_ssa_1e5_red = np.interp(mesh_dlr, mesh_ssa_1e5, P_sliced_ssa_1e5[-1][0])
P_sliced_ssa_1e6_red = np.interp(mesh_dlr, mesh_ssa_1e6, P_sliced_ssa_1e6[-1][0])
P_sliced_ssa_1e7_red = np.interp(mesh_dlr, mesh_ssa_1e7, P_sliced_ssa_1e7[-1][0])

fig, axs = plt.subplots(2, 2, figsize=(6, 5))
axs[0, 0].plot(mesh_dlr, P_sliced[0], "bx", label="DLR approx.")
axs[0, 0].plot(mesh_dlr, P_sliced_ssa_1e4_red, "gs", label="SSA")

axs[0, 1].plot(mesh_dlr, P_sliced[0], "bx", label="DLR approx.")
axs[0, 1].plot(mesh_dlr, P_sliced_ssa_1e5_red, "gs")

axs[1, 0].plot(mesh_dlr, P_sliced[0], "bx", label="DLR approx.")
axs[1, 0].plot(mesh_dlr, P_sliced_ssa_1e6_red, "gs")

axs[1, 1].plot(mesh_dlr, P_sliced[0], "bx", label="DLR approx.")
axs[1, 1].plot(mesh_dlr, P_sliced_ssa_1e7_red, "gs")

max_error_ssa_1e4 = np.max(np.abs(P_sliced_ssa_1e4_red - P_sliced[0]))
max_error_ssa_1e5 = np.max(np.abs(P_sliced_ssa_1e5_red - P_sliced[0]))
max_error_ssa_1e6 = np.max(np.abs(P_sliced_ssa_1e6_red - P_sliced[0]))
max_error_ssa_1e7 = np.max(np.abs(P_sliced_ssa_1e7_red - P_sliced[0]))

axs[0, 0].set_title("10\,000 runs", y=0.95)
axs[0, 1].set_title("100\,000 runs", y=0.95)
axs[1, 0].set_title("1\,000\,000 runs", y=0.95)
axs[1, 1].set_title("10\,000\,000 runs", y=0.95)

plt.setp(axs.flat, 
         yscale="log", 
         xlabel="$x_1$", 
         ylabel="$P_\mathrm{{S}}(x_1)$", 
         xlim=(9, 25),
         ylim=(1e-9, 1e-2)
         )

fig.tight_layout()
axs[0, 0].legend(fontsize="9")
fig.savefig('plots/bax_general_fig1.pgf')
fig.savefig('plots/bax_general_fig1.pdf')

## Figure 2
Two-dimensional sliced distribution $P_\mathrm{S}(x_1) = P(x_1, 9, 2, 1, 0, 0, 0, 0, 0, 50, 0)$ for $t = 145$

In [ ]:
result = np.load("scripts/reference_solutions/bax_ssa_general_ref_t145_1e4.npy")
ssa_sol_1e4 = SSASol(result)
_, _, _, P_sliced2D_ssa_1e4 = ssa_sol_1e4.calculateObservables(slice_vec, idx_2D)

result = np.load("scripts/reference_solutions/bax_ssa_general_ref_t145_1e5.npy")
ssa_sol_1e5 = SSASol(result)
_, _, _, P_sliced2D_ssa_1e5 = ssa_sol_1e5.calculateObservables(slice_vec, idx_2D)

result = np.load("scripts/reference_solutions/bax_ssa_general_ref_t145_1e6.npy")
ssa_sol_1e6 = SSASol(result)
_, _, _, P_sliced2D_ssa_1e6 = ssa_sol_1e6.calculateObservables(slice_vec, idx_2D)

result = np.load("scripts/reference_solutions/bax_ssa_general_ref_t145_1e7.npy")
ssa_sol_1e7 = SSASol(result)
_, _, _, P_sliced2D_ssa_1e7 = ssa_sol_1e7.calculateObservables(slice_vec, idx_2D)

In [ ]:
n_min = np.amax(np.stack((grid.liml, ssa_sol_1e4.n_min, ssa_sol_1e5.n_min, ssa_sol_1e6.n_min, ssa_sol_1e7.n_min)), axis=0)
n_max = np.amin(np.stack((grid.n + grid.liml, ssa_sol_1e4.n_max,
                ssa_sol_1e5.n_max, ssa_sol_1e6.n_max, ssa_sol_1e7.n_max)), axis=0)
n = n_max - n_min + 1
dx = np.prod(n)

P_sliced2D_red = np.zeros(n[:2])
P_sliced2D_ssa_1e4_red = np.zeros(n[:2])
P_sliced2D_ssa_1e5_red = np.zeros(n[:2])
P_sliced2D_ssa_1e6_red = np.zeros(n[:2])
P_sliced2D_ssa_1e7_red = np.zeros(n[:2])

liml_red = n_min - grid.liml
liml_red_ssa_1e4 = n_min - ssa_sol_1e4.n_min
liml_red_ssa_1e5 = n_min - ssa_sol_1e5.n_min
liml_red_ssa_1e6 = n_min - ssa_sol_1e6.n_min
liml_red_ssa_1e7 = n_min - ssa_sol_1e7.n_min

for i in range(n[0]):
    for j in range(n[1]):
        P_sliced2D_red[i, j] = P_sliced2D[i + liml_red[0], j + liml_red[1]]
        P_sliced2D_ssa_1e4_red[i, j] = P_sliced2D_ssa_1e4[-1][i +
                                                              liml_red_ssa_1e4[0] + ssa_sol_1e4.n[0] * (j + liml_red_ssa_1e4[1])]
        P_sliced2D_ssa_1e5_red[i, j] = P_sliced2D_ssa_1e5[-1][i +
                                                              liml_red_ssa_1e5[0] + ssa_sol_1e5.n[0] * (j + liml_red_ssa_1e5[1])]
        P_sliced2D_ssa_1e6_red[i, j] = P_sliced2D_ssa_1e6[-1][i +
                                                              liml_red_ssa_1e6[0] + ssa_sol_1e6.n[0] * (j + liml_red_ssa_1e6[1])]
        P_sliced2D_ssa_1e7_red[i, j] = P_sliced2D_ssa_1e7[-1][i +
                                                              liml_red_ssa_1e7[0] + ssa_sol_1e7.n[0] * (j + liml_red_ssa_1e7[1])]

In [ ]:
xx1, xx2 = np.meshgrid(np.arange(n[0]) + n_min[0], np.arange(n[1]) + n_min[1], indexing='ij')

fig, axs = plt.subplots(1, 5, figsize=(10, 8))
plt.setp(axs.flat, xlabel='$x_1$', ylabel='$x_2$')

vmax = np.amax([np.amax(P_sliced2D_red), np.amax(P_sliced2D_ssa_1e4_red), np.amax(P_sliced2D_ssa_1e5_red), np.amax(P_sliced2D_ssa_1e6_red), np.amax(P_sliced2D_ssa_1e7_red)])
vmin = np.amax([0.0, np.amin([np.amin(P_sliced2D_red), np.amin(P_sliced2D_ssa_1e4_red), np.amin(P_sliced2D_ssa_1e5_red), np.amin(P_sliced2D_ssa_1e6_red), np.amin(P_sliced2D_ssa_1e7_red)])])

norm = mpl.colors.LogNorm(vmin=vmin, vmax=vmax)
cmap = 'RdBu_r'

axs[0].imshow(P_sliced2D_red, norm=norm, cmap=cmap)
axs[1].imshow(P_sliced2D_ssa_1e4_red, norm=norm, cmap=cmap)
axs[2].imshow(P_sliced2D_ssa_1e5_red, norm=norm, cmap=cmap)
axs[3].imshow(P_sliced2D_ssa_1e6_red, norm=norm, cmap=cmap)
axs[4].imshow(P_sliced2D_ssa_1e7_red, norm=norm, cmap=cmap)

axs[0].set_title("DLR approx.")
axs[1].set_title("SSA (10\,000 runs)")
axs[2].set_title("SSA (100\,000 runs)")
axs[3].set_title("SSA (1\,000\,000 runs)")
axs[4].set_title("SSA (10\,000\,000 runs)")
fig.tight_layout()
fig.savefig('plots/bax_general_fig2.pgf', dpi=300)
fig.savefig('plots/bax_general_fig2.pdf', dpi=300)